<a href="https://colab.research.google.com/github/k-k1208/AI--quest/blob/master/vgg_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# class, functionの一覧
1. ImageTransform(class):画像の前処理クラス
  - 適応データ：train, val,test

2. make_datapath_list(function):画像へのファイルパスをリスト型変数に格納する関数
  - 適応データ：train, val , test

3. Dataset(HymenopteraDataset)(class):1,2のクラスを使って作成
  - 適応データ：train, val, test

    - Dataloader:datasetsからバッチごとに取り出すことを目的に使われる
      - 適応データ：train,valのみ

4. train_model(function):モデルを学習させる関数
  - 適応データ：train, val

5. test_model(function):学習モデルを使って予測する関数
  - 適応データ：test
  - memo：できれば、submitできる形式をreturnできるのが理想


In [1]:
#google driveをgoogle colabにマウントする
from google.colab import drive 
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd "/content/drive/My Drive/AI_QUEST"
%ls

/content/drive/My Drive/AI_QUEST
 output2.tsv   sample_submit.tsv                 test/    vgg_model.ipynb
 output.tsv   'signate_AI Quest アセスメント'/   train/


In [3]:
# パッケージのimport
import glob
import os.path as osp
import random
import numpy as np
import json
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchvision
from torchvision import models, transforms

In [4]:
# 乱数のシードを設定
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [5]:
# 入力画像の前処理をするクラス
# 訓練時と推論時で処理が異なる


class ImageTransform():
    """
    画像の前処理クラス。訓練時、検証時で異なる動作をする。
    画像のサイズをリサイズし、色を標準化する。
    訓練時はRandomResizedCropとRandomHorizontalFlipでデータオーギュメンテーションする。


    Attributes
    ----------
    resize : int
        リサイズ先の画像の大きさ。
    mean : (R, G, B)
        各色チャネルの平均値。
    std : (R, G, B)
        各色チャネルの標準偏差。
    """

    def __init__(self, resize, mean, std):
        self.data_transform = {
            'train': transforms.Compose([
                transforms.RandomResizedCrop(
                    resize, scale=(0.5, 1.0)),  # データオーギュメンテーション
                transforms.RandomHorizontalFlip(),  # データオーギュメンテーション
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'val': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ]),
            'test': transforms.Compose([
                transforms.Resize(resize),  # リサイズ
                transforms.CenterCrop(resize),  # 画像中央をresize×resizeで切り取り
                transforms.ToTensor(),  # テンソルに変換
                transforms.Normalize(mean, std)  # 標準化
            ])
        }

    def __call__(self, img, phase='train'):
        """
        Parameters
        ----------
        phase : 'train' or 'val'
            前処理のモードを指定。
        """
        return self.data_transform[phase](img)


In [ ]:
def make_datapath_list(phase="train"):
    """
    データのパスを格納したリストを作成する。

    Parameters
    ----------
    phase : 'train' or 'val'
        訓練データか検証データかを指定する

    Returns
    -------
    path_list : list
        データへのパスを格納したリスト
    """

    rootpath = "/content/drive/My Drive/AI_QUEST/"
    if phase == "train":
      target_path = osp.join(rootpath+phase+'/**/*.jpeg')
    elif phase == "test":
      target_path = osp.join(rootpath+phase+'/**.jpeg')
    else:
      print("ファイルパス読み取り時の予期せぬエラー")
    print(target_path)
    path_list = []  # ここに格納する

    # globを利用してサブディレクトリまでファイルパスを取得する
    for path in glob.glob(target_path):
        path_list.append(path)

    return path_list


# 実行
trainval_list = make_datapath_list(phase="train")
test_list = make_datapath_list(phase="test")

In [7]:
class HymenopteraDataset(data.Dataset):
    """
    アリとハチの画像のDatasetクラス。PyTorchのDatasetクラスを継承。

    Attributes
    ----------
    file_list : リスト
        画像のパスを格納したリスト
    transform : object
        前処理クラスのインスタンス
    phase : 'train' or 'test'
        train: trainとval
        test : public leader boardの予測用
    """

    def __init__(self, file_list, transform=None, phase='train'):
        self.file_list = file_list  # ファイルパスのリスト
        self.transform = transform  # 前処理クラスのインスタンス
        self.phase = phase  # train or valの指定

    def __len__(self):
        '''画像の枚数を返す'''
        return len(self.file_list)

    def __getitem__(self, index):
        '''
        前処理をした画像のTensor形式のデータとラベルを取得
        '''

        # index番目の画像をロード
        img_path = self.file_list[index]
        img = Image.open(img_path)  # [高さ][幅][色RGB]

        # 画像の前処理を実施
        img_transformed = self.transform(
            img, self.phase)  # torch.Size([3, 224, 224])
        if self.phase == 'train':
          # 画像のラベルをファイル名から抜き出す
          label = img_path[39:40]      

          # ラベルを数値に変更する
          # 1:不良品、0：良品
          if label == "b" or  label == "h" or label == "p":
              label = 1
          elif label == "r":
              label = 0
          else:
              label = 'ラベルerrorやで'

          return img_transformed, label
        
        elif self.phase == 'test':
          return img_transformed
        
        else:
          print('Dataset作成時の予期せぬエラー')


In [9]:
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
trainval_dataset = HymenopteraDataset(
    file_list=trainval_list, transform=ImageTransform(size, mean, std), phase='train')

test_dataset = HymenopteraDataset(
    file_list=test_list, transform=ImageTransform(size, mean, std), phase='test')

In [11]:
# アリとハチの画像へのファイルパスのリストを作成する
trainval_list = make_datapath_list(phase="train")
test_list = make_datapath_list(phase="test")

# Datasetを作成する
size = 224
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)
trainval_dataset = HymenopteraDataset(
    file_list=trainval_list, transform=ImageTransform(size, mean, std), phase='train')
test_dataset = HymenopteraDataset(
    file_list=test_list, transform=ImageTransform(size, mean, std), phase='test')


n_samples = len(trainval_dataset) # n_samples is 290
train_size = int(len(trainval_dataset) * 0.8)  #232
val_size = n_samples - train_size #58

# shuffleしてから分割してくれる.
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size, val_size])

# DataLoaderを作成する
batch_size = 32

train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)

val_dataloader = torch.utils.data.DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False)

test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=1, shuffle=False)


# 辞書オブジェクトにまとめる
dataloaders_dict = {"train": train_dataloader, "val": val_dataloader}

/content/drive/My Drive/AI_QUEST/train/**/*.jpeg
/content/drive/My Drive/AI_QUEST/test/**.jpeg


In [12]:
# 学習済みのVGG-16モデルをロード

# VGG-16モデルのインスタンスを生成
use_pretrained = True  # 学習済みのパラメータを使用
net = models.vgg16(pretrained=use_pretrained)

# VGG16の最後の出力層の出力ユニットをアリとハチの2つに付け替える
net.classifier[6] = nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()

print('ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました')

ネットワーク設定完了：学習済みの重みをロードし、訓練モードに設定しました


In [13]:
# 1.損失関数の設定
criterion = nn.CrossEntropyLoss()

# 2.最適化手法を設定
# ファインチューニングで学習させるパラメータを、変数params_to_updateの1～3に格納する

params_to_update_1 = []
params_to_update_2 = []
params_to_update_3 = []

# 学習させる層のパラメータ名を指定
update_param_names_1 = ["features"]
update_param_names_2 = ["classifier.0.weight",
                        "classifier.0.bias", "classifier.3.weight", "classifier.3.bias"]
update_param_names_3 = ["classifier.6.weight", "classifier.6.bias"]

# パラメータごとに各リストに格納する
for name, param in net.named_parameters():
    if update_param_names_1[0] in name:
        param.requires_grad = True
        params_to_update_1.append(param)
        print("params_to_update_1に格納：", name)

    elif name in update_param_names_2:
        param.requires_grad = True
        params_to_update_2.append(param)
        print("params_to_update_2に格納：", name)

    elif name in update_param_names_3:
        param.requires_grad = True
        params_to_update_3.append(param)
        print("params_to_update_3に格納：", name)

    else:
        param.requires_grad = False
        print("勾配計算なし。学習しない：", name)


params_to_update_1に格納： features.0.weight
params_to_update_1に格納： features.0.bias
params_to_update_1に格納： features.2.weight
params_to_update_1に格納： features.2.bias
params_to_update_1に格納： features.5.weight
params_to_update_1に格納： features.5.bias
params_to_update_1に格納： features.7.weight
params_to_update_1に格納： features.7.bias
params_to_update_1に格納： features.10.weight
params_to_update_1に格納： features.10.bias
params_to_update_1に格納： features.12.weight
params_to_update_1に格納： features.12.bias
params_to_update_1に格納： features.14.weight
params_to_update_1に格納： features.14.bias
params_to_update_1に格納： features.17.weight
params_to_update_1に格納： features.17.bias
params_to_update_1に格納： features.19.weight
params_to_update_1に格納： features.19.bias
params_to_update_1に格納： features.21.weight
params_to_update_1に格納： features.21.bias
params_to_update_1に格納： features.24.weight
params_to_update_1に格納： features.24.bias
params_to_update_1に格納： features.26.weight
params_to_update_1に格納： features.26.bias
params_to_update_1に格納： f

In [14]:
#最適化手法の設定
optimizer = optim.SGD([
    {'params': params_to_update_1, 'lr': 1e-4},
    {'params': params_to_update_2, 'lr': 5e-4},
    {'params': params_to_update_3, 'lr': 1e-3}
], momentum=0.9)


In [15]:
# モデルを学習させる関数を作成


def train_model(net, dataloaders_dict, criterion, optimizer, num_epochs):

    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True

    # epochのループ
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-------------')

        # epochごとの訓練と検証のループ
        for phase in ['train', 'val']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに
            else:
                net.eval()   # モデルを検証モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数

            # 未学習時の検証性能を確かめるため、epoch=0の訓練は省略
            if (epoch == 0) and (phase == 'train'):
                continue

            # データローダーからミニバッチを取り出すループ
            for inputs, labels in tqdm(dataloaders_dict[phase]):

                # GPUが使えるならGPUにデータを送る
                inputs = inputs.to(device)
                labels = labels.to(device)

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = net(inputs)
                    loss = criterion(outputs, labels)  # 損失を計算
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)

            # epochごとのlossと正解率を表示
            epoch_loss = epoch_loss / len(dataloaders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloaders_dict[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))


In [16]:
num_epochs=4
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)

使用デバイス： cuda:0


  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1/4
-------------


  0%|          | 0/8 [00:00<?, ?it/s]

val Loss: 0.6853 Acc: 0.5690
Epoch 2/4
-------------


...

val Loss: 0.5583 Acc: 0.7414
Epoch 4/4
-------------


  0%|          | 0/2 [00:00<?, ?it/s]

train Loss: 0.5956 Acc: 0.7198


100%|██████████| 2/2 [00:12<00:00,  6.33s/it]

val Loss: 0.4666 Acc: 0.7759


In [ ]:
net

In [17]:
# モデルをtestさせる関数を作成


def test_model(net, datasets):
    # 初期設定
    # GPUが使えるかを確認
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)

    # ネットワークをGPUへ
    net.to(device)

    net.eval()   # モデルを検証モードに
    preds = []
    labels = []
    # データローダーからミニバッチを取り出すループ
    for inputs in datasets:

        # GPUが使えるならGPUにデータを送る
        inputs = inputs.to(device)

        # 順伝搬（forward）計算
        # with torch.set_grad_enabled(phase == 'train'): # 学習時にのみ勾配の算出を有効にする設定です
        outputs = net(inputs)
        
        _, pred = torch.max(outputs, 1)  # ラベルを予測
        preds.append(pred)
        for i in range(len(preds)):
          label = preds[i][0].item()
          labels.append(label)

    return labels

In [19]:
lablels = test_model(net, test_dataloader)

使用デバイス： cuda:0


In [ ]:
labels 

In [27]:
file_names = []
tsv_list = []

for i in range(len(test_list)):
  file_name = test_list[i][38:]
  file_names.append(file_name)

for name, label in zip(file_names, labels):
  tsv_list.append([name, label])

tsv_list = sorted(tsv_list) #リストをimageファイル001から数字順にsort

In [29]:
#提出ファイル(tsv形式)の作成
import csv

# 新規作成専用でファイルオープン: 引数mode='x' :ファイルが存在しない場合のみ書き込み（新規作成のみ）
# 書き込み用でファイルオープン: 引数mode='w'
with open('output3.tsv', 'x') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    for row in tsv_list:
      tsv_writer.writerow(row)
